In [6]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message


def error_handler(msg):

    print("Server Error: %s" % msg)

def reply_handler(msg):

    print("Server Response: %s, %s" % (msg.typeName, msg))

In [10]:
tws_conn = Connection.create("127.0.0.1", port=7497, clientId=999)
tws_conn.connect()
tws_conn.register(error_handler, 'Error')  
tws_conn.registerAll(reply_handler)

Server Version: 76
TWS Time at connection:20181218 07:06:28 EST


True

Server Response: managedAccounts, <managedAccounts accountsList=DU1300822>
Server Response: nextValidId, <nextValidId orderId=1>
Server Error: <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm.nj>
Server Response: error, <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm.nj>
Server Error: <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture>
Server Response: error, <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture>
Server Error: <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
Server Response: error, <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
Server Error: <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
Server Response: error, <error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
Server Error: <error id=-1, errorCode=2106, errorMsg=HMDS

In [11]:
order_id = 1
goog_contract = create_contract('GOOG', 'STK', 'SMART', 'SMART', 'USD')
goog_order = create_order('MKT', 100, 'BUY')

In [13]:
tws_conn.placeOrder(order_id, goog_contract, goog_order)


Server Response: openOrder, <openOrder orderId=1, contract=<ib.ext.Contract.Contract object at 0x7f26341839e8>, order=<ib.ext.Order.Order object at 0x7f2634183d30>, orderState=<ib.ext.OrderState.OrderState object at 0x7f26342c1198>>
Server Response: orderStatus, <orderStatus orderId=1, status=Submitted, filled=0, remaining=100, avgFillPrice=0.0, permId=1710808424, parentId=0, lastFillPrice=0.0, clientId=999, whyHeld=None>
Server Response: execDetails, <execDetails reqId=-1, contract=<ib.ext.Contract.Contract object at 0x7f26342c1198>, execution=<ib.ext.Execution.Execution object at 0x7f2634183940>>
Server Response: openOrder, <openOrder orderId=1, contract=<ib.ext.Contract.Contract object at 0x7f2634183940>, order=<ib.ext.Order.Order object at 0x7f26342c1198>, orderState=<ib.ext.OrderState.OrderState object at 0x7f26341a20b8>>
Server Response: orderStatus, <orderStatus orderId=1, status=Filled, filled=100, remaining=0, avgFillPrice=1036.28, permId=1710808424, parentId=0, lastFillPrice=

In [9]:
tws_conn.disconnect()

True